# NASA: Asteroids Classification

### Overview

We were provided the NASA asteroids classification [dataset](https://www.kaggle.com/datasets/lovishbansal123/nasa-asteroids-classification). The dataset contains information about near-Earth asteroids, including features such as diameter, orbital period, semi-major axis, and whether they are hazardous.

### Project Objectives

1. **Identify Hazardous Asteroids**: Use the AI system to classify asteroids.
2. **Feature Analysis**: Identify key features for classification.
3. **Model Comparison**: Compare different machine learning algorithms.


In [ ]:
# Import all nessesary libraries
import datetime
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
from joblib import dump, load
from tqdm import tqdm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_predict, cross_validate, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Set warnings 
warnings.filterwarnings('ignore')

## Data processing

### Data loading

In [ ]:
path = 'data/nasa.csv'
data = pd.read_csv(path)
data.head()

In [ ]:
data.info()
data['Hazardous'].value_counts()

### Missing Values

Check for Missing Values (NA) in a DataFrame:

In [ ]:
print(f"Is any data NA: {data.isna().any().any()}")

### Visualizing Correlation Matrix Using Heatmap

The purpose of this code snippet is to create a heatmap using seaborn to visualize the correlation matrix of relationships between different numerical columns in the dataset. Using that we can drop the unnecessary columns to optimize our models or analysis.

In [ ]:
# plt.figure(figsize=(32,32))
# sb.heatmap(data.corr(numeric_only=True), annot=True, fmt='.2f')
# plt.show()

### Analyzing High Correlations in Columns

Based on the results, several columns have high correlations:

1. **Name** and **Neo Reference ID**: These are just identifiers and do not contain useful information for analysis.

2. **Est Dia in M(min)**, **Est Dia in Miles(min)** ... : All these values represent the same measurements but in different units. We can drop all but one.

3. **Relative Velocity km per hr** and **Miles per hour**: Similar to the previous case, these values represent the same information but in different units.

4. **Miss Dist.(Astronomical)** and **Miss Dist.(lunar)**: These also represent the same information but in different units.

5. **Orbital Period**: This has a perfect correlation (1.0) with **Semi Major Axis**, due to the formula $T^2 = \text{const} \cdot a^3$, where $T$ is the Orbital Period and $a$ is the Semi Major Axis. We can drop one of them.

6. **Est Dia in KM(max)**, **Jupiter Tisserand Invariant**, **Epoch Osculation**, **Aphelion Dist**: These columns have high correlations with other values.

Additionally, we can drop the columns **Equinox** and **Orbiting Body**, as they contain only a single unique value: *J2000* and *Earth*, respectively.


In [ ]:
print(f"Unique Equinox values: {data['Equinox'].nunique()}")
print(f"Unique Orbiting Body values: {data['Orbiting Body'].nunique()}")

data = data.drop(['Name', 'Neo Reference ID', 'Orbit ID',   # IDs
                    'Orbit Determination Date', 'Close Approach Date', # Date
                    'Est Dia in M(min)', 'Est Dia in M(max)', 'Est Dia in Miles(min)', 'Est Dia in Miles(max)', 'Est Dia in Feet(min)', 'Est Dia in Feet(max)', # Distance
                    'Relative Velocity km per hr', 'Miles per hour', # Velocity
                    'Miss Dist.(Astronomical)', 'Miss Dist.(lunar)', 'Miss Dist.(miles)', # # Distance
                    'Equinox',            # all 'J2000' 
                    'Orbiting Body',      # all 'Earth'
                    'Orbital Period',     # T² = const * a³, => corr = 1 with Semi Major Axis (a)
                    'Est Dia in KM(max)', # High correlation with Est Dia in KM(min)
                    'Jupiter Tisserand Invariant', # High correlation with Mean Motion
                    'Epoch Osculation',   # High correlation with Perihelion Time
                    'Aphelion Dist',      # High correlation with Semi Major Axis
                    ] , axis = 1)

data['Hazardous'] = data['Hazardous'].astype(int)

data.head()

New Correlation Matrix:

In [ ]:
# plt.figure(figsize=(20,20))
# sb.heatmap(data.corr(numeric_only=True), annot=True, fmt='.2f')
# plt.show()

### Dropping outliers

The goal of this section is to identify and potentially remove outliers from the dataset. 

Plotting Pairwise Relationships:

In [ ]:
# plt.figure()
# sb.pairplot(data.dropna(), hue='Hazardous')
# plt.show()

Filter out outliers based on specific conditions, based on the estimated diameter (min) in kilometers, semi-major axis, and inclination, keeping only reasonable values.

In [ ]:
# Filter out outliers based on specific conditions
df_no_outliers = data[(data['Est Dia in KM(min)'] <= 10) & 
                    (data['Semi Major Axis'] <= 4) &
                    (data['Inclination'] < 70)]

# Reset the index of the filtered DataFrame
df_no_outliers.reset_index(inplace=True, drop=True)

print(f"Number of objects before: {data.shape[0]}, after: {df_no_outliers.shape[0]}, dropped: {data.shape[0] - df_no_outliers.shape[0]} objects.")

In [ ]:
# plt.figure()
# sb.pairplot(df_no_outliers.dropna(), hue='Hazardous')
# plt.show()

### Save clean data

In [ ]:
clean_data_path = 'data/nasa_clean.csv'
print(f"Saving data to {clean_data_path} ...")
df_no_outliers.to_csv(clean_data_path, index=False)

## Models training

### Preparation

Separate the features (X) and the target variable (y)

In [ ]:
X = df_no_outliers.drop(['Hazardous'], axis=1)
y = df_no_outliers['Hazardous']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

Scale the numerical features of the dataset to have zero mean and unit variance. This standardization helps to improve the performance of many machine learning algorithms.

In [ ]:
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer([
    ("num", numerical_transformer, X.columns.to_list()),
])

preprocessor.fit(X_train)

x_train_transformed = preprocessor.transform(X_train)
x_test_transformed = preprocessor.transform(X_test)

print("Transformed data:")
pd.DataFrame(x_train_transformed, columns=X.columns).head()

In [ ]:
def plot_metric_resuts(df_results, names):
    scores = ['Acc Test', 'Pre Test', 'Rec Test', 'F1 Test']
    heatmap_data = df_results[scores].transpose()

    plt.figure(figsize=(12, 6))
    ax = sb.heatmap(heatmap_data, annot=True, cmap='Blues', cbar=True, xticklabels=names, yticklabels=scores)
    ax.set_title("Model Performance Heatmap (Test Set)")
    ax.set_xlabel("Models")
    ax.set_ylabel("Metrics")
    plt.tight_layout()
    plt.show()

def plot_boxplots(results_acc, results_pre, results_rec, results_f1, names):
    # Create boxplots
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 6))

    plot_titles = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    results_list = [results_acc, results_pre, results_rec, results_f1]

    palette = sb.color_palette("Blues")
    for ax, title, result in zip(axes, plot_titles, results_list):
        sb.boxplot(data=result, ax=ax, palette=palette)
        ax.set_title(title, fontsize=14)
        ax.set_xticklabels(names, rotation=30)

    plt.tight_layout()
    plt.show()

def plot_confusion_matrices(models, confusion_matrices, names):
    # Create Confusion Matrices
    fig, axes = plt.subplots(1, len(models), figsize=(5 * len(models), 4))
    if len(models) == 1:
        axes = [axes]
    for ax, cm, name in zip(axes, confusion_matrices, names):
        sb.heatmap(cm / np.sum(cm), annot=True, fmt=".2%", ax=ax, cmap='Blues')
        ax.set_title(f'{name} Confusion Matrix')
        ax.set_xlabel('Predicted labels')
        ax.set_ylabel('True labels')
    plt.tight_layout()
    plt.show()

In [ ]:
def model_evaluation(x_train, y_train, x_test, y_test, models):
    names = []
    scoring = ['accuracy', 'precision', 'recall', 'f1']
    
    # Create a dataframe to store the different metric values for each algorithm
    df_results = pd.DataFrame(columns=['Algorithm', 'Acc Train Mean', 'Acc Train STD', 'Pre Train Mean', 'Pre Train STD', 
                                       'Rec Train Mean', 'Rec Train STD', 'F1 Train Mean', 'F1 Train STD',
                                       'Acc Test', 'Pre Test', 'Rec Test', 'F1 Test', 'Training Time'])
    results_acc_test = []
    results_pre_test = []
    results_rec_test = []
    results_f1_test = []
    results_acc = []
    results_pre = []
    results_rec = []
    results_f1 = [] 
    confusion_matrices = []
    
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    for name, model in tqdm(models, total=len(models)):
        names.append(name)

        # Cross-validation results on the training set
        result = cross_validate(model, x_train, y_train, cv=kfold, scoring=scoring, return_train_score=False)
        model.fit(x_train, y_train)
        
        # Test set results
        predictions = model.predict(x_test)
        cm = confusion_matrix(y_test, predictions)
        confusion_matrices.append(cm)
        accuracy_test = accuracy_score(y_test, predictions)
        precision_test = precision_score(y_test, predictions, zero_division=1)
        recall_test = recall_score(y_test, predictions)
        f1_test = f1_score(y_test, predictions)
        
        # Store the test set results (single values)
        results_acc_test.append(accuracy_test)
        results_pre_test.append(precision_test)
        results_rec_test.append(recall_test)
        results_f1_test.append(f1_test)
        results_acc.append(result['test_accuracy'])
        results_pre.append(result['test_precision'])
        results_rec.append(result['test_recall'])
        results_f1.append(result['test_f1'])

        # Row of results
        model_results = pd.DataFrame({'Algorithm': name, 
                                      'Acc Test': accuracy_test,
                                      'Pre Test': precision_test,
                                      'Rec Test': recall_test,
                                      'F1 Test': f1_test,
                                      'Acc Train Mean': result['test_accuracy'].mean(), 
                                      'Acc Train STD': result['test_accuracy'].std(), 
                                      'Pre Train Mean': result['test_precision'].mean(), 
                                      'Pre Train STD': result['test_precision'].std(), 
                                      'Rec Train Mean': result['test_recall'].mean(), 
                                      'Rec Train STD': result['test_recall'].std(), 
                                      'F1 Train Mean': result['test_f1'].mean(), 
                                      'F1 Train STD': result['test_f1'].std(),
                                      'Training Time': result['fit_time'].mean(),
                                      }, index=[0])
        
        # Add the row to the results data frame
        if not model_results.empty:
            df_results = pd.concat([df_results, model_results], ignore_index=True)
        
        # Save model.
        current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        # dump(model, f'models/{name}_{current_time}_classifier.joblib')


    df_results = df_results.set_index('Algorithm')
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    pd.set_option('display.width', 1000)
    print(df_results)


    # Show different plots
    plot_metric_resuts(df_results, names)
    plot_boxplots(results_acc, results_pre, results_rec, results_f1, names)
    plot_confusion_matrices(models, confusion_matrices, names)

In [ ]:
models = []
models.append(('SVC', SVC()))
models.append(('DTC', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('GNB', GaussianNB()))
models.append(('XGB', XGBClassifier()))
models.append(('MLP', MLPClassifier()))

model_evaluation(x_train_transformed, y_train, x_test_transformed, y_test, models)

In [ ]:
models = []
models.append(('RFC', RandomForestClassifier()))
models.append(('ABC', AdaBoostClassifier(algorithm='SAMME')))
models.append(('GBC', GradientBoostingClassifier()))
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('LGBM', LGBMClassifier(verbose=-1)))


model_evaluation(x_train_transformed, y_train, x_test_transformed, y_test, models)

### Grid search

In this section, we'll use the Grid Search algorithm to improve the performance of two models: the best and worst models from our previous evaluations.

* Best model: [RandomForestClassifire](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* Worst model: [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_features': ['auto', 'sqrt', 'log2'], 
    'max_depth': [None, 5, 10, 20, 30],  
    'min_samples_split' : [1, 2, 3],
    'criterion': ['gini', 'entropy']  
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)

grid_search.fit(x_train_transformed, y_train)

best_rf = grid_search.best_estimator_

predictions = best_rf.predict(x_test_transformed)

accuracy = accuracy_score(y_test, predictions)

print("Best parameters found: ", grid_search.best_params_)
print("Test accuracy of the best model: ", accuracy)

In [ ]:
knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15],  
    'weights': ['uniform', 'distance'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 
    'leaf_size': [15, 20, 30, 40], 
    'p': [1, 1.25, 1.5, 1.75, 2]  
}

grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)

grid_search.fit(x_train_transformed, y_train)

best_knn = grid_search.best_estimator_

predictions = best_knn.predict(x_test_transformed)
accuracy = accuracy_score(y_test, predictions)

print("Best parameters found: ", grid_search.best_params_)
print("Test accuracy of the best model: ", accuracy)